In [1]:
import pybryt
from lecture import pybryt_reference

# Introduction to Python

# Lecture 5

## Introduction

As we saw in [lecture 4](..\lecture4\lecture4.ipynb), Python classes combine a storing data (attributes) and functions (methods) that process that data. They act as a way to create new data types. In the previous lecture, we learned some of the individual features of classes. In this section we will learn to put multiple features together to implement and improve a class based on the functionality we need it to provide.

## Example 1: Quaternions

Just as complex numbers extend the set of real numbers into a "two-dimensional" space, [quaternions](https://en.wikipedia.org/wiki/Quaternion) extend the concept of "2D" complex numbers into a "four-dimensional" space. Quaternions are used in computer graphics, control theory, signal processing, and orbital mechanics, and more generally in areas involving applying multiple rotations or scalings using floating point arthimetic. In the realm of science, they are also used in physics to describe the quantum spin of a particle. We are going to briefly describe the mathematical properties of quaternions as numbers and then you will work to implement a class to represent them.

### Basic Definition

A quarterions is an object (viewed as a single number) consisting four real coefficients, say $a$, $b$, $c$, and $d$, along with rules for addition, mutilplication by a scalar, and multiplication by another quaternion. The rules are as follows:

1. Addition: 
  $$(a, b, c, d) + (e, f, g, h) = (a+e, b+f, c+g, d+h)$$
2. Multiplication by a scalar:
  $$r(a, b, c, d) = (ra, rb, rc, rd)$$
3. Multiplication by another quaternion: 
  $$(a, b, c, d)(e, f, g, h) = (ae - bf - cg - dh, af + be + ch - dg, ag - bh + ce + df, ah + bg - cf + de)$$
These rules are enough to define the quaternions as a number system, as well as a [vector space](https://en.wikipedia.org/wiki/Vector_space) over the real numbers.

Just as a complex number can be written in the form $a + bi$ (or in Python in a form like `1+2j`), a quaternion can be written in the form $a + bi + cj + dk$. This means they keep both engineers and mathematicians happy by having `i` & `j` both denote complex components of the number.

By the rules of multiplication above we have:
$$ i^2 = j^2 = k^2 = ijk = -1, $$
$$ ij = k, \quad jk = i, \quad ki = j. $$
In this sense, we can think of the $i$, $j$ and $k$ as three separate complex dimensions.

### Implementing Quaternions

This is enough information to start implementing a class to represent quaternions. We'll do this in a single cell immediately below. As you work through this section, we'll add more features to the class, and need to re-run the cell to update the class definition. If anything doesn't work or your results get confusing, try restarting the Python kernel and re-running that cell to see it it cleans up the problem.

#### The Class Definition cell

In [19]:
# Remember to re-run this cell after you've modified the class below
# Carry on reading to see what to implement.
import numpy as np


class Quaternion:

    def __init__(self, a, b, c, d):
        """Initializes a quaternion with the given values.

        The first value is the real part, the rest are the imaginary parts. The imaginary
        parts are i, j, and k.  For example, Quaternion(1, 2, 3, 4) represents the
        quaternion 1 + 2i + 3j + 4k.

        """
        # Although there are no private attributes in Python, by convention, we use an
        # underscore to indicate that an attribute is private and should not be accessed
        # directly. This is a convention, and it is not enforced by the language.
        
        self._1 = a
        self._i = b
        self._j = c
        self._k = d

    def __str__(self):
        """Returns the string representation of the quaternion."""

        # Helper function to format components
        def format_component(value, suffix):
            return f" {'+' if value >= 0 else '-'} {abs(value)}{suffix}"

        # Format each component
        components = [
            f"{self._1}",
            format_component(self._i, "i"),
            format_component(self._j, "j"),
            format_component(self._k, "k"),
        ]

        return "".join(components)
    
    def __add__(self, other):
        
        """Adds two quaternions component-wise."""
        if not isinstance(other, Quaternion):
            return NotImplemented
        return Quaternion(
            self._1 + other._1,
            self._i + other._i,
            self._j + other._j,
            self._k + other._k,
        )
    
    def __mul__(self, other):
        if isinstance(other, (int, float)):  # scalar multiplication
            return Quaternion(
                self._1 * other, self._i * other, self._j * other, self._k * other
            )
        elif isinstance(other, Quaternion):  # Hamilton product
            a1, b1, c1, d1 = self._1, self._i, self._j, self._k # i got confused so i had to make them a, b, c, d
            a2, b2, c2, d2 = other._1, other._i, other._j, other._k
            return Quaternion(
                a1*a2 - b1*b2 - c1*c2 - d1*d2,
                a1*b2 + b1*a2 + c1*d2 - d1*c2,
                a1*c2 - b1*d2 + c1*a2 + d1*b2,
                a1*d2 + b1*c2 - c1*b2 + d1*a2
            )
        else:
            return NotImplemented

        # for rmul, i used a commutator to swap between them
        
    def commutator(self, other):
        """Return [self, other] = self*other - other*self"""
        if not isinstance(other, Quaternion):
            raise TypeError("Commutator is only defined between two quaternions")
        pq = self * other
        qp = other * self
        return Quaternion(
            pq._1 - qp._1,
            pq._i - qp._i,
            pq._j - qp._j,
            pq._k - qp._k,
        )

    def __rmul__(self, other):
        if isinstance(other, (int, float)):  # scalar * quaternion
            return Quaternion(
                other * self._1,
                other * self._i,
                other * self._j,
                other * self._k,
            )
        elif isinstance(other, Quaternion):  # right multiplication via commutator
            pq = self * other
            comm = self.commutator(other)
            return Quaternion(
                pq._1 - comm._1,
                pq._i - comm._i,
                pq._j - comm._j,
                pq._k - comm._k,
            )
        else:
            return NotImplemented
        
    def conjugate(self):
        """Return the conjugate of the quaternion: a - bi - cj - dk"""
        return Quaternion(self._1, -self._i, -self._j, -self._k)
    
    def __abs__(self):
        """Return the magnitude of the quaternion."""
        return np.sqrt(self._1**2 + self._i**2 + self._j**2 + self._k**2)

    def inverse(self):
        """Return the inverse of the quaternion."""
        norm_sq = abs(self) ** 2
        if norm_sq == 0:
            raise ZeroDivisionError("Cannot invert a quaternion with zero magnitude")
        conj = self.conjugate()
        return Quaternion(
            conj._1 / norm_sq,
            conj._i / norm_sq,
            conj._j / norm_sq,
            conj._k / norm_sq,
        )

As with most classes, the first thing to do is to implement the initialisation method so that we can create new instances of the class from appropriate information. In this case, we've done this for you, and chosen the names of the attributes to be `_1`, `_i`, `_j`, and `_k`.

We've also implemented the `__str__` method, which is called when the object is printed (as well as a few other times). This is a good way to check that the initialisation method is working as expected, and we can use it to test the core class initialisation.

In [20]:
assert str(Quaternion(1, 2, 3, 4)) == "1 + 2i + 3j + 4k"

The cell block above will show no output if the class is implemented correctly. If you see a `NameError`, then make sure that you've run the cell defining the `Quaternion` class. If you see an `AssertionError`, then
make sure you haven't changed the initialisation or `__str__` method.

### Implementing the Addition Method

With the code we've already given you, you'll first need to implement the addition and multiplication methods. The addition method name to use the `+` operator is `__add__`, and it takes two arguments. The first argument is always the object itself (and as with the initialisation and `__str__` methods, it is good practice to use `self` to refer to it). The second argument is the other object being added - the Python convention is to call it `other`. This means the method signature is

```python
   def __add__(self, other):
     # method body ending with a return statement
```

Add this method to the Quaterion class above, and implement the addition rule

$$a+b\mathit{i}+c\mathit{j}+d\mathit{k} + e+f\mathit{i}+g\mathit{j}+h\mathit{k} = a+e+(b+f)\mathit{i}+(c+g)\mathit{j}+(d+h)\mathit{k}.$$

Once you're done, we can test the method by adding two quaternions together (Remember to re-run the class cell above after you've made changes to the class definition). If you see a `TypeError`, then check you've added the `__add__` method correctly (and rerun the class cell). One common mistake is geting the wrong amount of white space indentation. If you see an `AssertError` then your returned value isn't right.

In [21]:
q1 = Quaternion(1, 2, 3, 4)
q2 = Quaternion(5, 6, 7, 8)

assert str(q1 + q2) == "6 + 8i + 10j + 12k", "Got: " + str(q1 + q2)

### Implementing the Multiplication Method

Now, we want to implement multiplication. The method name is `__mul__`, and it takes the same arguments as the addition method. This means that method signature is

```python
   def __mul__(self, other):
     # method body ending with a return statement
```

Here the method body will need to be a little more complicated, since there are two possible cases:
- If `other` is a scalar number, then we need to multiply each component of the quaternion by `other`.
- If `other` is another quaternion, then we need to use the multiplication rule above.

We can check the type of `other` using the `isinstance` function. This function takes two arguments, the first is the object to check, and the second is the type to check against. It returns `True` if the object is of the given type, and `False` otherwise. For example, `isinstance(3, int)` returns `True`, and `isinstance(3, float)` returns `False`. In this case, we want to check if the `other` is a `Quaternion` object or not, using `isinstance(other, Quaternion)`.

The "quick" way to implement scalar multiplication is just to code up the multiplication rule for quaternions, and if `other` is not a `Quaternion` object, then convert it into one with the scalar as the first component, i.e. creating `Quaternion(other, 0, 0, 0)`. Alternatively, you can use a set of `if`, `elif` and (if necessary) `else` statements to handle the cases separately. In the first case, your code will look something like

```python
def __mul__(self, other):
   if not isinstance(other, Quaternion):
      other = Quaternion(other, 0, 0, 0)
  # rest of the method body    
```

in the second case, your code will look something like

```python
def __mul__(self, other):
   if isinstance(other, Quaternion):
      # result = multiplication rule for quaternion
      return result
   else:
      # multiplication rule for scalar
      return result
```
 
Try to implement all this, and then run our test cases below.

In [22]:
q1 = Quaternion(1, 2, 3, 4)
q2 = Quaternion(5, 6, 7, 8)

assert str(q1 * q2) == "-60 + 12i + 30j + 24k", "Got: " + str(q1 * q2)
assert str(q2 * q1) == "-60 + 20i + 14j + 32k", "Got: " + str(q2 * q1)
assert str(q1 * 3) == "3 + 6i + 9j + 12k", "Got: " + str(q1 * 3)

### Additional Methods

We've now implemented a method to multiply "from the left", i.e. solving `q * other`, but for the purposes of scalar multiplication, we also need to implement a method to multiply "from the right", i.e. solving `other * q`.

The appropriate special method (also called a "magic method" or "dunder method") name is `__rmul__`, and it takes the same arguments as the multiplication method, except this time "other" is the object on the left. This means that method signature is

```python
   def __rmul__(self, other):
     # method body ending with a return statement
```

Note that we can't just call `return self.__mul__(other)` here, since for quaternions, multiplication is not commutative (i.e. `q1 * q2` is not equal to `q2 * q2` for general quaternions `q1` and `q2`). Again there are several ways to solve this, but the one involving least code is to make `other` into a `Quaternion` object if it isn't already, and then call its `__mul__` method on self (either explicity as `other.__mul__(self)` or implicitly through `other*self`).



In [23]:
q1 = Quaternion(1, 2, 3, 4)

assert str(3 * q1) == "3 + 6i + 9j + 12k", "Got: " + str(3 * q1)

We've now implemented the core arthmetic operations for quaternions, and can move on to more advanced features. Let's start by defining the conjugate of a quaternion. Just as the conjugate of a complex number $z= a+b\mathit{i}$ is $\bar{z} = a - b\mathit{i}$,  the conjugate of a quaternion $q = a + b\mathit{i} + c\mathit{j} + d\mathit{k}$ is defined as $\bar{q} = a - b\mathit{i} - c\mathit{j} - \mathit{k}$. 

We can implement this as a method `conjugate` that takes no additional arguments, and returns a new quaternion object with the "imaginary" parts negated. The method signature should be

```python
   def conjugate(self):
     # method body ending with a return statement
```

Note that $q\bar{q} := \|q\|^2 = a^2 + b^2 + c^2 + d^2$, which is a positive real number. We can define the inverse or reciprocal of a quaternion as $\frac{\bar{q}}{\|q\|^2}$, and implement this as a method `inverse`.
```python
   def inverse(self):
     # method body ending with a return statement
```
It's not technically correct to call this division, due to the non-commutative nature of quaternion multiplication. This means that there are two "inverse functions" for multiplication, one which solves $xa = b$ for $x$ and one which solves $ax = b$. Using the notation $a^{-1}$ for the inverse above, the inverse from the right is $ba^{-1}$ and the inverse is $a^{-1}b$.

Instead of computing the magnitude $|q|$ inside the `inverse` method, implement the special `__abs__` method. This method will be called when built in `abs` function is called on a Quaternion object. Its signature is:
```python
   def __abs____(self):
     # method body ending with a return statement
```

Try to implement the `conjugate` and `inverse` methods in your `Quaternion` class above, and then run the test cases below.

In [24]:
q1 = Quaternion(1, 1, 1, 1)

assert str(q1.conjugate()) == "1 - 1i - 1j - 1k", "Got: " + str(q1.conjugate())
assert str(abs(q1)) == "2.0", "Got: " + str(abs(q1))
assert str(q1.inverse()) == "0.25 - 0.25i - 0.25j - 0.25k", "Got: " + str(q1.inverse())

### Refactoring the class (optional)

We've now implemented all the basic functionality of quaternions. However, the class is not particularly user-friendly. Here is a list of some things you could improve:

- The `__init__` method is currently not very user-friendly. It would be nice if we could create a quaternion in a few extra ways:
  - Let `Quaternion(1) == Quaternion(1, 0, 0, 0)`.
  - Let `Quaternion(1+2j) == Quaternion(1, 2, 0, 0)`.
  - Let `Quaternion([1, 2, 3, 4]) == Quaternion(1, 2, 3, 4)`.
  
This can be done in a number of ways. One lower effort solution is to use the `*args` and/or `**kwargs` syntax to allow for a variable number of arguments and keyword arguments. 

In [25]:
def function_with_var_args(*args, **kwargs):
    print(args, kwargs)


function_with_var_args(1, 2)
function_with_var_args(1, 2, 3, 4, 5)
function_with_var_args(1, 2, 3, 4, 5, a=6, b=7, c=8)

(1, 2) {}
(1, 2, 3, 4, 5) {}
(1, 2, 3, 4, 5) {'a': 6, 'b': 7, 'c': 8}


Another things we can implement are:

- We have implemented the `__str__` method, but not the `__repr__` method. The `__repr__` method is called when the object is printed in a list or dictionary, and should return a string that can be used to recreate the object. For example, `repr(Quaternion(1, 2, 3, 4))` should return the string `"Quaternion(1, 2, 3, 4)"`. If we copy/paste the output of `__repr__`, we should get the same object.
- We have implemented the `__add__`, `__mul__` and  `__rmul__`, methods, but not the `__sub__`, `__eq__` methods (or a few others).


In [26]:
assert repr(Quaternion(1, -2, 3, 4)) == "Quaternion(1, -2, 3, 4)"

assert Quaternion(1) == Quaternion(1, 0, 0, 0)
assert Quaternion(1, 2) == Quaternion(1, 2, 0, 0)
assert Quaternion(b=2) == Quaternion(0, 2, 0, 0)

assert Quaternion(1, 2, 3, 4) - Quaternion(5, 6, 7, 8) == Quaternion(-4, -4, -4, -4)

AssertionError: 

In [27]:
with pybryt.check(pybryt_reference(5, 1)):
    q1 = Quaternion(1, -2, 3, -4)
    q2 = Quaternion(15, 2, 0, -1)

    repr(q1), str(q1)
    repr(q1 + q2)
    repr(q2 - q1)
    repr(5 * q1)
    repr(q1 * q2)
    abs(q1)
    repr(q1.conjugate())

REFERENCE: exercise-5_1
SATISFIED: False
MESSAGES:
  - ERROR: It looks like the output of your repr method is not correct.
  - SUCCESS: Great! Your str method is correctly implemented.
  - ERROR: The output of your __add__ method is not as expected.
  - ERROR: The output of your __sub__ method is not correct.
  - ERROR: Your implementation of multiplication with a scalar is not correct.
  - ERROR: The output of your __mul__ method is not correct.
  - ERROR: The output of your __abs__ method is not correct.
  - ERROR: The conjugate of a quaternion is not correct.


TypeError: unsupported operand type(s) for -: 'Quaternion' and 'Quaternion'

## Example 2: Building Triangles

### Problem Statement

The second example we will consider is building a collection of classes to represent the information present in triangles. We've defined a class for points in two dimensions. You'll then use this to define a class for line segments (defined by two points), and then a class for triangles (defined by three points). We'll then add methods to calculate the area and perimeter of the triangle, and the length of the line segment.

### The Point Class Defintion

 We'll define a class to represent a point in 2D space. You will then need to implement a line segment (to represent an edge for a triangle), and finally a triangle. We will add appropriate methods to each class to calculate the area and perimeter of a triangle and the length of a line segment.

In [28]:
from dataclasses import dataclass


@dataclass
class Point:
    x: float
    y: float

Here we've used the `dataclass` decorator from the `dataclasses` module to define the class. This is a more concise way to define a class, and is particularly useful when the class is just a container for data.

The `dataclass` decorator automatically adds an `__init__` method, and `__repr__` method, and `__eq__` method, and a few other things. The `__init__` method takes the same arguments as the class, and the `__repr__` method returns a string that can be used to recreate the object. The `__eq__` method checks if two objects are equal. 

In [29]:
# Examples of usage
p1 = Point(1.0, 2.0)
p2 = Point(x=2.5, y=4.0)

print(p1, p2)
print(p1 == p2)
print(p1 == Point(1.0, 2.0))

Point(x=1.0, y=2.0) Point(x=2.5, y=4.0)
False
True


### The LineSegment Class Implementation

Given the `Point` class definition above, we can now implement a `LineSegment` class which takes two `Point` objects as arguments to its `__init__` method. The `__init__` method should store these two points as attributes of the object. We'll also want a `length` method to calculate the length of the line segment.

In [39]:
class LineSegment:
    def __init__(self, start: Point, end: Point):
        self.start = start
        self.end = end

    def __repr__(self):
        return f"LineSegment({self.start}, {self.end})"

    def length(self) -> float:
        dx = self.end.x - self.start.x
        dy = self.end.y - self.start.y
        return math.sqrt(dx**2 + dy**2)
    
    def midpoint(self) -> Point:
        """Return the midpoint as a Point object."""
        mx = (self.start.x + self.end.x) / 2
        my = (self.start.y + self.end.y) / 2
        return Point(mx, my)

    def intersects(self, other: "LineSegment") -> bool:
        """Return true if this segment intersects with another line segment."""

        # Represent each line in parametric form:
        # r1(t1) = p1 + t1*(p2 - p1)
        # r2(t2) = q1 + t2*(q2 - q1)
        p1, p2 = self.start, self.end
        q1, q2 = other.start, other.end

        # Direction vectors
        d1x, d1y = p2.x - p1.x, p2.y - p1.y
        d2x, d2y = q2.x - q1.x, q2.y - q1.y

        # Solve system:
        # p1 + t1*d1 = q1 + t2*d2
        # => t1*d1x - t2*d2x = q1.x - p1.x
        #    t1*d1y - t2*d2y = q1.y - p1.y
        A = [[d1x, -d2x],
             [d1y, -d2y]]
        b = [q1.x - p1.x, q1.y - p1.y]

        # Compute determinant
        det = A[0][0]*A[1][1] - A[0][1]*A[1][0]
        if abs(det) < 1e-10:
            # Lines are parallel (no intersection assumed)
            return False

        # Solve using Cramer's rule
        det_t1 = b[0]*A[1][1] - A[0][1]*b[1]
        det_t2 = A[0][0]*b[1] - b[0]*A[1][0]

        t1 = det_t1 / det
        t2 = det_t2 / det

        # Intersection only if both parameters are between 0 and 1
        return 0 <= t1 <= 1 and 0 <= t2 <= 1

Here we've used the optional "type hints" feature of Python to specify the types of the arguments to the `__init__` method. This is not necessary (nor it is checked), but can be useful for documentation and for catching errors. The type hints are specified in the method signature, and are separated from the argument name by a colon. For example, `p1: Point` specifies that `p1` is a `Point` object. Similarly, `-> float` suggests that the `length` method returns a `float` object. Once again, this is not checked anywhere, and only provides a hint to the user, either in code or through documentation.

We'll also need some tests on the class:

In [40]:
import math
from math import isclose

p1 = Point(0, 0)
p2 = Point(3, 4)

assert isclose(LineSegment(p1, p2).length(), 5)

#### Extending the LineSegment Class


- add a method to the `LineSegment` class called `midpoint()`. It should return a `Point` object representing the midpoint of the line segment.
- (more challenging) add a method to the `LineSegment` class called `intersects`. It should take another `LineSegment` object as an argument, and return `True` if the two line segments intersect, and `False` otherwise. You can assume that the line segments are not colinear if you'd like to reduce your workload.
  As a hint, you can use the following algorithm to check if two line segments intersect:
    - We can write any line segment as a parametric equation, i.e. $\mathbf{r}(t) = \mathbf{p}_1 + t(\mathbf{p}_2 - \mathbf{p}_1)$, where $\mathbf{p}_1$ and $\mathbf{p}_2$ are beginning and end of the line segment, and $t$ is a real number between 0 and 1.
    - Two line segments intersect if and only if the parametric equations of the two line segments have a common solution, i.e. there exists $t_1$ and $t_2$ such that $\mathbf{r}_1(t_1) = \mathbf{r}_2(t_2)$.
    - Substituting the parametric equations into the equation $\mathbf{r}_1(t_1) = \mathbf{r}_2(t_2)$, we get a system of 2 linear equations in $t_1$ and $t_2$. If the system has a solution (and the values of $t_1$ and $t_2$ are in the range $0-1$), then the line segments intersect.

In [41]:
p1 = Point(-2, -2)
p2 = Point(2, 2)

assert LineSegment(p1, p2).midpoint() == Point(0, 0)

line_segment1 = LineSegment(Point(0, 0), Point(10, 10))
line_segment2 = LineSegment(Point(1, 1), Point(11, 11))
line_segment3 = LineSegment(Point(10, 0), Point(0, 10))
line_segment4 = LineSegment(Point(0, 0), Point(10, 10))

assert not line_segment1.intersects(line_segment2)
assert line_segment1.intersects(line_segment3)
assert line_segment3.intersects(line_segment4)

In [42]:
with pybryt.check(pybryt_reference(5, 2)):
    l1 = LineSegment(Point(0, 0), Point(1, 1))
    l2 = LineSegment(Point(0, 1), Point(1, 0))

    repr(l1)
    l1.length()
    l2.midpoint().x
    l2.midpoint().y
    l1.intersects(l2)

REFERENCE: exercise-5_2
SATISFIED: True
MESSAGES:
  - SUCCESS: Your repr method is correctly implemented. Well done!
  - SUCCESS: Your length method is correct.
  - SUCCESS: Your midpoint calculation (x) is correct.
  - SUCCESS: Your midpoint calculation (y) is correct.
  - SUCCESS: Your intersection calculation seems to be correct.


### The Triangle class Implementation

Finally, we need to implement a `Triangle` class. We'll choose to use an initialisation method that takes three `Point` objects as arguments, and stores them as attributes of the object. We'll also want methods to calculate the area and perimeter of the triangle.

In [43]:
import math

class Triangle:
    def __init__(self, p1: Point, p2: Point, p3: Point):
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3

    def __repr__(self):
        return f"Triangle({self.p1}, {self.p2}, {self.p3})"

    def edges(self) -> tuple:
        """Return the three edges as LineSegments, ordered around the triangle."""
        e1 = LineSegment(self.p1, self.p2)
        e2 = LineSegment(self.p2, self.p3)
        e3 = LineSegment(self.p3, self.p1)
        return (e1, e2, e3)

    def perimeter(self) -> float:
        """Return the perimeter of the triangle."""
        return sum(edge.length() for edge in self.edges())

    def area(self) -> float:
        """Return the area using the shoelace formula."""
        x1, y1 = self.p1.x, self.p1.y
        x2, y2 = self.p2.x, self.p2.y
        x3, y3 = self.p3.x, self.p3.y
        return abs((x1*(y2-y3) + x2*(y3-y1) + x3*(y1-y2)) / 2.0)

    def angles(self) -> tuple:
        """Return the three interior angles in radians."""
        e1, e2, e3 = self.edges()

        # Side lengths
        a = e1.length()  # opposite to p1
        b = e2.length()  # opposite to p2
        c = e3.length()  # opposite to p3

        # Law of cosines
        angle1 = math.acos((b**2 + c**2 - a**2) / (2*b*c))
        angle2 = math.acos((a**2 + c**2 - b**2) / (2*a*c))
        angle3 = math.acos((a**2 + b**2 - c**2) / (2*a*b))

        return (angle1, angle2, angle3)

    def contains_point(self, pt: Point) -> bool:
        """Return True if pt lies inside or on the triangle, False otherwise."""
        verts = [self.p1, self.p2, self.p3]
        inside = True
        on_edge = False

        for i in range(3):
            x1, y1 = verts[i].x, verts[i].y
            x2, y2 = verts[(i+1)%3].x, verts[(i+1)%3].y

            # Edge vector normal
            nx, ny = (y2 - y1, x1 - x2)

            # Vector from vertex to point
            vx, vy = pt.x - x1, pt.y - y1

            dot = nx*vx + ny*vy
            if dot < -1e-10:  # tolerance for any error
                inside = False
            elif abs(dot) <= 1e-10:
                on_edge = True

        return inside or on_edge


In [44]:
p1 = Point(0, 0)
p2 = Point(1, 0)
p3 = Point(0, 1)

t = Triangle(p1, p2, p3)

assert isclose(t.area(), 0.5), "Check your area calculation"
assert isclose(t.perimeter(), 1 + 1 + math.sqrt(2)), "Check your perimeter calculation"
assert isclose(t.edges()[0].length(), 1) # made small change to specify .length() instead of the LineSegment object itself
assert isclose(t.edges()[1].length(), math.sqrt(2))
assert isclose(t.edges()[2].length(), 1)
assert isclose(t.angles()[0], math.pi / 4)
assert isclose(t.angles()[1], math.pi / 2)
assert isclose(t.angles()[2], math.pi / 4)
assert not t.contains_point(Point(1, 1))

### Extending the Triangle Class

- add a new method to the `Triangle` class, `edges()`, that returns a `tuple` containing the three `LineSegment`s which form the edges of the `Triangle`. 
The edges should be ordered so that the first vertex of the first edge is the first vertex of the triangle, and the second vertex of the first edge is the second vertex of the triangle, as well as being the first vertex of the second edge, and so on.
- add a method to the `Triangle` class called  `angles()`. It should return a tuple of three floats, each representing the interior angle _in radians_ of the vertices of the triangle, ordered so that the first angle is at the first vertex of the triangle, and so on.
- (more challenging) add a method to the `Triangle` class called `contains_point` that takes a `Point` object as an argument, and returns `True` if the point is inside (or on an edge of) the triangle, and `False` otherwise. You can use the following algorithm to solve this problem:
  - For each edge of the triangle, calculate the normal vector to the edge.   you can do this by calculating
    $$\mathbf{n} = \begin{pmatrix} y_2 - y_1 \\ x_1 - x_2 \end{pmatrix},$$
    where $(x_1, y_1)$ and $(x_2, y_2)$ are the coordinates of the two vertices of the edge.
  - Calculate the vector from one of the vertices of the edge to the point.
  - If the dot product of these two vectors is positive for all edges, then the point is inside the triangle. If the dot product is negative for any edge, then the point is outside the triangle. If the dot product is zero for an edge, (and positive for the others) then the point is on the edge of the triangle.
  
  You can use additional packages such as  `numpy`if you like, but it is not necessary.


In [45]:
p1 = Point(0, 0)
p2 = Point(1, 0)
p3 = Point(0, 1)

t = Triangle(p1, p2, p3)

assert isclose(t.edges()[0].length(), 1)
assert isclose(t.edges()[1].length(), math.sqrt(2))
assert isclose(t.edges()[2].length(), 1)
assert isclose(t.angles()[0], math.pi / 4)
assert isclose(t.angles()[1], math.pi / 2)
assert isclose(t.angles()[2], math.pi / 4)
assert not t.contains_point(Point(1, 1))

### Extending the shape design via inheritance (optional)

- We can also extend functionality to allow for more shapes, such as squares, hexagons and so on. The "best" way to do this in Python is to define a new "abstract" class called `Polygon` that implements reusable methods for generic shapes (where possible) in terms of the Points or edges of the polygon, and then define a new class for each shape that inherits from `Polygon` as a base class and which implements the methods that are specific to that shape (for example the `area` method for a triangle is different to the `area` method for a square).

By defining `Polygon` as 

In [46]:
from abc import ABC, abstractmethod


class Polygon(ABC):
    def __init__(self, vertices):
        # Write appropriate code here, accepting a collection
        # of Points as arguments, a representing a vertex of the polygon.
        pass

    def perimeter(self):
        # Write appropriate code here, returning the perimeter of the polygon
        # based on the stored vertices.
        pass

    def edges(self):
        # Write appropriate code here, returning a tuple of LineSegments
        # representing the edges of the polygon
        pass

    @abstractmethod
    def area(self):
        # Write appropriate code here, returning the area of the polygon
        raise NotImplementedError

In [38]:
class Quadrilateral(Polygon):
    def __init__(self, a, b, c, d):
        # Here, super() will call the __init__ method of the parent class -
        # Polygon.__init__().
        super().__init__(a, b, c, d)

    def area(self):
        # Write appropriate code here, returning the area of the quadrilateral.
        raise NotImplementedError

## Example 3: Designing your own class

### Problem Statement

The final problem is always the most challenging: designing your own class from scratch. We'll give you a problem statement, and you'll need to design a class to solve it. The problem statement is as follows:

Build a class to represent the data provided from a [weather station](https://en.wikipedia.org/wiki/Weather_station). The station has the following fixed data attached to it:
- A name (a string)
- A location (a tuple of two floats, representing the latitude and longitude)
- A height above sea level (a float, representing the height in meters)

The station also has (at a minimum) the following data that change over time:

- A temperature (a float, representing the temperature in degrees Celsius)
- A humidity (a float, representing the humidity as a percentage)
- The pressure (a float, representing the pressure in hPa)

The class should have methods to do the following:

- An initialisation method (`__init__`) that takes the name, location, and height above sea level as arguments, and stores them, as well as doing some initialisation of the temperature, humidity and pressure (plus other variables if you've added them).
- A method to update the temperature, humidity, and pressure.
- Appropriately implemented `__str__` and `__repr__` methods.
- Methods to calculate appropriate "weather" metrics, such as the [dew point](https://en.wikipedia.org/wiki/Dew_point) (it's ok to use the simplified formula), the [heat index](https://en.wikipedia.org/wiki/Heat_index), and so on, and to convert between different temperature scales (e.g. Celsius, Fahrenheit, Kelvin). You may want to make use of the `@property` decorator to make these methods feel like attributes to the user.

In [47]:
# Write your class here. As with previous examples, you may
# want to test aspects of your class as you go along

class WeatherStation:
    def __init__(self, name: str, location: tuple[float, float], height: float):
        self.name = name
        self.location = location  # (latitude, longitude)
        self.height = height      # meters above sea level

        # Initialize dynamic measurements (set to None until updated)
        self.temperature = None  # °C
        self.humidity = None     # %
        self.pressure = None     # hPa

    @property
    def temp_celsius(self) -> float:
        """Return the temperature in Celsius."""
        return self.temperature
    
    @property
    def temp_fahrenheit(self) -> float:
        """Return the temperature in Fahrenheit."""
        if self.temperature is None:
            return None
        return self.temperature * 9/5 + 32
    
    @property
    def temp_kelvin(self) -> float:
        """Return the temperature in Kelvin."""
        if self.temperature is None:
            return None
        return self.temperature + 273.15
    
    @property
    def dew_point(self) -> float:
        """Return the dew point temperature in Celsius."""
        if self.temperature is None or self.humidity is None:
            return None

        a, b = 17.27, 237.7
        alpha = (a * self.temperature) / (b + self.temperature) + math.log(self.humidity / 100.0)
        return (b * alpha) / (a - alpha)
    
    @property
    def heat_index(self) -> float:
        """Return the heat index ("feels like" temperature) in Celsius,
        using the NOAA regression formula.
        """
        if self.temperature is None or self.humidity is None:
            return None

        # Convert to Fahrenheit for formula
        T = self.temperature * 9/5 + 32
        RH = self.humidity

        # NOAA regression constants, i did not know there were this many
        c1 = -42.379
        c2 = 2.04901523
        c3 = 10.14333127
        c4 = -0.22475541
        c5 = -6.83783e-3
        c6 = -5.481717e-2
        c7 = 1.22874e-3
        c8 = 8.5282e-4
        c9 = -1.99e-6

        HI = (c1 + (c2 * T) + (c3 * RH) + (c4 * T * RH) +
              (c5 * T**2) + (c6 * RH**2) + (c7 * T**2 * RH) +
              (c8 * T * RH**2) + (c9 * T**2 * RH**2))

        # Convert back to Celsius
        return (HI - 32) * 5/9
    
    @property
    def vapor_pressure(self) -> float:
        """Return the vapor pressure in hPa based on temperature and relative humidity."""
        if self.temperature is None or self.humidity is None:
            return None

        # Saturation vapor pressure (Magnus formula)
        T = self.temperature  # °C
        e_s = 6.112 * math.exp((17.67 * T) / (T + 243.5))

        # Actual vapor pressure
        return (self.humidity / 100.0) * e_s
    
    @property
    def relative_humidity(self) -> float:
        """Return relative humidity (%) based on temperature and vapor pressure."""
        if self.temperature is None or self.vapor_pressure is None:
            return None

        # Saturation vapor pressure at current temperature
        T = self.temperature
        e_s = 6.112 * math.exp((17.67 * T) / (T + 243.5))

        # RH as %
        return (self.vapor_pressure / e_s) * 100

    def __repr__(self):
        return (f"WeatherStation(name={self.name!r}, "
                f"location={self.location}, "
                f"height={self.height}, "
                f"temperature={self.temperature}, "
                f"humidity={self.humidity}, "
                f"pressure={self.pressure})")

    def __str__(self):
        return (f"Weather Station '{self.name}' at {self.location}, {self.height} m asl\n"
                f"Temp: {self.temperature} °C, "
                f"Humidity: {self.humidity} %, "
                f"Pressure: {self.pressure} hPa")


In [48]:
ws = WeatherStation("Mountain Station", (40.7812, -73.9665), 2000.0)
ws.temperature = 32.0
ws.humidity = 57.0
ws.pressure = 800.0

print(ws)

Weather Station 'Mountain Station' at (40.7812, -73.9665), 2000.0 m asl
Temp: 32.0 °C, Humidity: 57.0 %, Pressure: 800.0 hPa


In [50]:
from math import isclose

# Create a test weather station
ws = WeatherStation("Test Station", (0.0, 0.0), 0.0)

# Set base measurements
ws.temperature = 25.0  # °C
ws.humidity = 60.0     # %

# --- Dew Point ---
expected_dew_point = 16.7
assert isclose(ws.dew_point, expected_dew_point, rel_tol=1e-2), f"Dew point failed: {ws.dew_point}"

# --- Heat Index ---
ws.temperature = 30.0
ws.humidity = 70.0
expected_heat_index = 35.0
assert isclose(ws.heat_index, expected_heat_index, rel_tol=1e-2), f"Heat index failed: {ws.heat_index}"

# --- Vapor Pressure ---
ws.temperature = 25.0
ws.humidity = 60.0
expected_vapor_pressure = 6.112 * (math.exp((17.67 * ws.temperature)/(ws.temperature + 243.5))) * (ws.humidity / 100)
assert isclose(ws.vapor_pressure, expected_vapor_pressure, rel_tol=1e-2), f"Vapor pressure failed: {ws.vapor_pressure}"

# --- Relative Humidity ---
# Should match the stored humidity since we computed vapor_pressure from it
assert isclose(ws.relative_humidity, ws.humidity, rel_tol=1e-2), f"Relative humidity failed: {ws.relative_humidity}"

# --- Temperature conversions ---
ws.temperature = 20.0
assert isclose(ws.temp_celsius, 20.0, rel_tol=1e-6), f"Temp Celsius failed: {ws.temp_celsius}"
assert isclose(ws.temp_fahrenheit, 68.0, rel_tol=1e-6), f"Temp Fahrenheit failed: {ws.temp_fahrenheit}"
assert isclose(ws.temp_kelvin, 293.15, rel_tol=1e-6), f"Temp Kelvin failed: {ws.temp_kelvin}"

print("its all worked, very cool")


its all worked, very cool
